# Scratch
This is where I keep scratch code snippets that may come in handy elsewhere.

## Convert Paros pressure to depth by fitting to the Sentry depth

### Load data and slice nav by depth

In [ ]:
# load converted data
import pandas as pd
nav = pd.read_hdf('../data/interim/nav_converted.h5', 'table')
paros = pd.read_hdf('../data/interim/paros_converted.h5', 'table')

In [ ]:
nav.head(1)

In [ ]:
paros.head(1)

In [ ]:
import numpy as np
depth_min = int(np.floor(nav.depth.min()))
depth_max = int(np.ceil(nav.depth.max()))
hist, edges = np.histogram(nav.depth.values, bins=np.arange(depth_min, depth_max))

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

p = figure(tools='pan,wheel_zoom,box_zoom,reset', x_range=(-1550, -1450), y_range=(0, 6e5),
          title='Histogram of Sentry Depth')
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color='blue', line_color=None)
show(p)

In [ ]:
nav_slice = nav.loc[(nav.depth >= -1550) & (nav.depth <= -1450)].copy() # here copy() is required to tell Pandas that we really do want a new copy of the dataframe, to avoid the SettingWithCopy warning
nav_slice.head(1)

### Interpolate Paros pressure onto nav_slice

In [ ]:
nav_slice['pressure'] = np.interp(nav_slice.epoch.values, paros.epoch.values, paros.pressure.values)
nav_slice.head(1)

### Least squares regression using Numpy polyfit

In [ ]:
params = np.polyfit(nav_slice.pressure.values, nav_slice.depth.values, 1)
print('Fit Parameters: %0.10f %0.10f' % (params[0], params[1]))

In [ ]:
nav_slice['paros_depth'] = nav_slice.pressure*params[0] + params[1]
nav_slice.head(1)

### Plot the Sentry depth and the Paros depth for a single dive

In [ ]:
p = figure(tools='pan,wheel_zoom,box_zoom,reset', title='asdf')
dive = 272
nav_dive = nav_slice.loc[(nav_slice.dive_number == dive)].copy()
p.line(nav_dive.epoch, nav_dive.paros_depth, line_color='blue')
p.line(nav_dive.epoch, nav_dive.depth, line_color='green')
show(p)

## Misc

In [ ]:
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()


p = figure(width=800, height=400, x_axis_type="datetime")
#p.line(ustrain_adv.iloc[::100].index.values, ustrain_adv.iloc[::100, 0])
show(p)

In [ ]:
%matplotlib notebook
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib as mpl

from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

In [ ]:
scc = store.scc.sort_values(by='T1')
norm = mpl.colors.Normalize(vmin=2.45, vmax=2.6)
cmap = mpl.cm.plasma
m = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
colors = ['#%02x%02x%02x' % (int(r),int(g),int(b)) for r,g,b,_ in 255*m.to_rgba(scc['T1'])]

# create a new figure
p = figure(tools='pan,wheel_zoom,box_zoom,reset', match_aspect=True, output_backend="webgl", lod_threshold=100)

# add a circle renderer with vecorized colors and sizes
p.circle(scc['x'], scc['y'], radius=1, fill_color=colors, fill_alpha=0.6, line_color=None)

# show the results
show(p)

In [ ]:
type((scc['T1']-scc['T1'].min())/scc['T1'].max())

In [ ]:
list(colors)[0]

In [ ]:
len(colors)

In [ ]:
mpl.cm.viridis(3)

In [ ]:
norm = mpl.colors.Normalize(vmin=2.4, vmax=2.65)
cmap = mpl.cm.hot
m = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
colors = list(m.to_rgba(scc['T1']))

#### Show rotated survey lines

In [ ]:
# subsample the vehicle locations to make a smaller plot
x = nav.iloc[::10].x
y = nav.iloc[::10].y
xrot = nav.iloc[::10].xrot
yrot = nav.iloc[::10].yrot

In [ ]:
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

# create a new figure
p = figure(tools='pan,wheel_zoom,box_zoom,reset', match_aspect=True, output_backend="webgl", lod_threshold=100)

# add a circle renderer with vecorized colors and sizes
shiftx = -15
shifty = -680

p.circle(x-x.min()+shiftx, y-y.min()+shifty, radius=1, fill_color='blue', fill_alpha=0.6, line_color=None)
p.circle(xrot-xrot.min()+shiftx, yrot-yrot.min()+shifty, radius=1, fill_color='green',
         fill_alpha=0.6, line_color=None)

# show the results
show(p)

#### Build a rotated coordinate system
Here we create a rotated coordinate system which may make gridding easier. We also shift the arbitrary coordinate system (meters) so that the lower left corner of the study area is at approximately (0,0).

In [ ]:
import numpy as np
omega = 29 # rotate by omega degrees
theta = np.deg2rad(omega)
v = np.transpose(np.array([nav.x.values, nav.y.values]))
R = np.array([[np.cos(theta),-np.sin(theta)], [np.sin(theta),np.cos(theta)]])
vr = v.dot(R)
nav['xrot'] = vr[:,0] - vr[:,0].min() - 615
nav['yrot'] = vr[:,1] - vr[:,1].min() - 1503

#### Interpolate nav onto all sensor data and save
We delete variables and free memory as we go to avoid running out of memory.

In [ ]:
paros['x'] = np.interp(paros['epoch'].values, nav['epoch'].values, nav['x'].values)
paros['y'] = np.interp(paros['epoch'].values, nav['epoch'].values, nav['y'].values)
paros['xrot'] = np.interp(paros['epoch'].values, nav['epoch'].values, nav['xrot'].values)
paros['yrot'] = np.interp(paros['epoch'].values, nav['epoch'].values, nav['yrot'].values)

In [ ]:
sbe3['x'] = np.interp(sbe3['epoch'].values, nav['epoch'].values, nav['x'].values)
sbe3['y'] = np.interp(sbe3['epoch'].values, nav['epoch'].values, nav['y'].values)
sbe3['xrot'] = np.interp(sbe3['epoch'].values, nav['epoch'].values, nav['xrot'].values)
sbe3['yrot'] = np.interp(sbe3['epoch'].values, nav['epoch'].values, nav['yrot'].values)

In [ ]:
nortek = pd.read_hdf('../data/interim/nortek_raw.h5', 'table')
nortek['x'] = np.interp(nortek['epoch'].values, nav['epoch'].values, nav['x'].values)
nortek['y'] = np.interp(nortek['epoch'].values, nav['epoch'].values, nav['y'].values)
nortek['xrot'] = np.interp(nortek['epoch'].values, nav['epoch'].values, nav['xrot'].values)
nortek['yrot'] = np.interp(nortek['epoch'].values, nav['epoch'].values, nav['yrot'].values)

In [ ]:
ustrain_adv = pd.read_hdf('../data/interim/ustrain_adv_raw.h5', 'table')
ustrain_adv = ustrain_adv.rename(index=str, columns={'x': 'xx', 'y': 'yy', 'z': 'zz'})
ustrain_adv['x'] = np.interp(ustrain_adv['epoch'].values, nav['epoch'].values, nav['x'].values)
ustrain_adv['y'] = np.interp(ustrain_adv['epoch'].values, nav['epoch'].values, nav['y'].values)
ustrain_adv['xrot'] = np.interp(ustrain_adv['epoch'].values, nav['epoch'].values, nav['xrot'].values)
ustrain_adv['yrot'] = np.interp(ustrain_adv['epoch'].values, nav['epoch'].values, nav['yrot'].values)

In [ ]:
ustrain_chassis = pd.read_hdf('../data/interim/ustrain_chassis_raw.h5', 'table')
ustrain_chassis = ustrain_chassis.rename(index=str, columns={'x': 'xx', 'y': 'yy', 'z': 'zz'})
ustrain_chassis['x'] = np.interp(ustrain_chassis['epoch'].values, nav['epoch'].values, nav['x'].values)
ustrain_chassis['y'] = np.interp(ustrain_chassis['epoch'].values, nav['epoch'].values, nav['y'].values)
ustrain_chassis['xrot'] = np.interp(ustrain_chassis['epoch'].values, nav['epoch'].values, nav['xrot'].values)
ustrain_chassis['yrot'] = np.interp(ustrain_chassis['epoch'].values, nav['epoch'].values, nav['yrot'].values)